In [2]:
# Pip install method (recommended)
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.6/8062.4 GB disk)


In [3]:
import os # operating system

In [4]:
mkdir /kaggle/working/datasets

In [5]:
mkdir /kaggle/working/datasets/train

In [6]:
mkdir /kaggle/working/datasets/val

In [7]:
mkdir /kaggle/working/datasets/test

In [8]:
mkdir /kaggle/working/datasets/train/images

In [9]:
mkdir /kaggle/working/datasets/train/annotations

In [10]:
mkdir /kaggle/working/datasets/val/images

In [11]:
mkdir /kaggle/working/datasets/val/annotations

In [12]:
mkdir /kaggle/working/datasets/test/images

In [13]:
mkdir /kaggle/working/datasets/test/annotations

In [14]:
from distutils.dir_util import copy_tree

In [15]:
cd /kaggle

/kaggle


In [16]:
# Copy train images from kaggle input directory to kaggle output directory
fromDir = '/kaggle/input/visdrone2019-det-train/train/images'
toDirectory = '/kaggle/working/datasets/train/images'
copy_tree(fromDir, toDirectory)

# Copy val images from kaggle input directory to kaggle output directory
fromDir = '/kaggle/input/visdrone2019-det-val/val/images'
toDirectory = '/kaggle/working/datasets/val/images'
copy_tree(fromDir, toDirectory)

# Copy test images from kaggle input directory to kaggle output directory
fromDir = '/kaggle/input/visdrone2019-det-test-dev/images'
toDirectory = '/kaggle/working/datasets/test/images'
copy_tree(fromDir, toDirectory)

['/kaggle/working/datasets/test/images/9999973_00000_d_0000028.jpg',
 '/kaggle/working/datasets/test/images/9999938_00000_d_0000034.jpg',
 '/kaggle/working/datasets/test/images/0000063_05000_d_0000006.jpg',
 '/kaggle/working/datasets/test/images/9999979_00000_d_0000012.jpg',
 '/kaggle/working/datasets/test/images/9999973_00000_d_0000083.jpg',
 '/kaggle/working/datasets/test/images/9999938_00000_d_0000124.jpg',
 '/kaggle/working/datasets/test/images/9999952_00000_d_0000050.jpg',
 '/kaggle/working/datasets/test/images/0000320_00150_d_0000003.jpg',
 '/kaggle/working/datasets/test/images/9999996_00000_d_0000017.jpg',
 '/kaggle/working/datasets/test/images/9999952_00000_d_0000140.jpg',
 '/kaggle/working/datasets/test/images/9999938_00000_d_0000363.jpg',
 '/kaggle/working/datasets/test/images/9999952_00000_d_0000192.jpg',
 '/kaggle/working/datasets/test/images/9999952_00000_d_0000049.jpg',
 '/kaggle/working/datasets/test/images/9999952_00000_d_0000169.jpg',
 '/kaggle/working/datasets/test/im

In [17]:
# Copy train annotations from kaggle input directory to kaggle output directory
fromDir = '/kaggle/input/visdrone2019-det-train/train/annotations'
toDirectory = '/kaggle/working/datasets/train/annotations'
copy_tree(fromDir, toDirectory)

# Copy val annotations from kaggle input directory to kaggle output directory
fromDir = '/kaggle/input/visdrone2019-det-val/val/annotations'
toDirectory = '/kaggle/working/datasets/val/annotations'
copy_tree(fromDir, toDirectory)

# Copy test annotations from kaggle input directory to kaggle output directory
fromDir = '/kaggle/input/visdrone2019-det-test-dev/annotations'
toDirectory = '/kaggle/working/datasets/test/annotations'
copy_tree(fromDir, toDirectory)

['/kaggle/working/datasets/test/annotations/9999952_00000_d_0000200.txt',
 '/kaggle/working/datasets/test/annotations/0000011_01350_d_0000003.txt',
 '/kaggle/working/datasets/test/annotations/9999938_00000_d_0000253.txt',
 '/kaggle/working/datasets/test/annotations/9999938_00000_d_0000019.txt',
 '/kaggle/working/datasets/test/annotations/0000074_11362_d_0000022.txt',
 '/kaggle/working/datasets/test/annotations/9999952_00000_d_0000053.txt',
 '/kaggle/working/datasets/test/annotations/9999938_00000_d_0000195.txt',
 '/kaggle/working/datasets/test/annotations/9999973_00000_d_0000060.txt',
 '/kaggle/working/datasets/test/annotations/9999993_00000_d_0000022.txt',
 '/kaggle/working/datasets/test/annotations/9999993_00000_d_0000016.txt',
 '/kaggle/working/datasets/test/annotations/9999963_00000_d_0000074.txt',
 '/kaggle/working/datasets/test/annotations/9999952_00000_d_0000118.txt',
 '/kaggle/working/datasets/test/annotations/9999973_00000_d_0000050.txt',
 '/kaggle/working/datasets/test/annota

In [18]:
from pathlib import Path

train: "train/images" # train images (relative to 'path')  6471 images
val: "val/images" # val images (relative to 'path')  548 images
test: "test/images" # test images (optional)  1610 images
dir = Path("working/datasets")  # dataset root dir
print(dir)

def visdrone2yolo(dir):
    from PIL import Image
    from tqdm import tqdm

    def convert_box(size, box):
        # Convert VisDrone box to YOLO xywh box
        dw = 1. / size[0]
        dh = 1. / size[1]
        return (box[0] + box[2] / 2) * dw, (box[1] + box[3] / 2) * dh, box[2] * dw, box[3] * dh

    (dir / 'labels').mkdir(parents=True, exist_ok=True)  # make labels directory
    pbar = tqdm((dir / 'annotations').glob('*.txt'), desc=f'Converting {dir}')
    for f in pbar:
        img_size = Image.open((dir / 'images' / f.name).with_suffix('.jpg')).size
        lines = []
        with open(f, 'r') as file:  # read annotation.txt
            for row in [x.split(',') for x in file.read().strip().splitlines()]:
                if row[4] == '0':  # VisDrone 'ignored regions' class 0
                    continue
                cls = int(row[5]) - 1
                box = convert_box(img_size, tuple(map(int, row[:4])))
                lines.append(f"{cls} {' '.join(f'{x:.6f}' for x in box)}\n")
                with open(str(f).replace(f'{os.sep}annotations{os.sep}', f'{os.sep}labels{os.sep}'), 'w') as fl:
                    fl.writelines(lines)  # write label.txt

working/datasets


In [19]:
for d in 'train', 'val', 'test':
      visdrone2yolo(dir / d)  # convert VisDrone annotations to YOLO labels

Converting working/datasets/train: 6471it [00:37, 173.99it/s]
Converting working/datasets/val: 548it [00:04, 127.90it/s]
Converting working/datasets/test: 1610it [00:08, 197.97it/s]


In [20]:
cd /kaggle/working/

/kaggle/working


In [21]:
# %rm -rf /kaggle/working/runs/detect

In [27]:
!yolo task=detect mode=train model=yolov8n.pt data=/kaggle/input/config/VisDrone.yaml epochs=50 imgsz=640 batch=32

New https://pypi.org/project/ultralytics/8.2.35 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/config/VisDrone.yaml, epochs=50, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_widt

In [28]:
!yolo task=detect mode=val model=runs/detect/train3/weights/best.pt data=/kaggle/input/config/VisDrone.yaml 

Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients
val: Scanning /kaggle/working/datasets/val/labels.cache... 548 images, 0 backgro
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        548      38759      0.357      0.263      0.251      0.141
            pedestrian        548       8844      0.377       0.25      0.249     0.0995
                people        548       5125      0.421      0.192      0.207     0.0687
               bicycle        548       1287      0.153     0.0404     0.0323      0.012
                   car        548      14064      0.571      0.682      0.682      0.443
                   van        548       1975      0.364      0.332      0.292      0.194
                 truck        548        750      0.386      0.229       0.22      0.141
              tricycle        548       1045      0.32

In [29]:
!yolo task=detect mode=predict model=runs/detect/train3/weights/best.pt source="/kaggle/input/visdrone2019-det-test-dev/images"

Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients

image 1/1610 /kaggle/input/visdrone2019-det-test-dev/images/0000006_00159_d_0000001.jpg: 384x640 25 cars, 3 vans, 3 trucks, 1 bus, 1 motor, 18.0ms
image 2/1610 /kaggle/input/visdrone2019-det-test-dev/images/0000006_00611_d_0000002.jpg: 384x640 2 pedestrians, 1 people, 31 cars, 2 vans, 3 trucks, 1 bus, 2 motors, 7.9ms
image 3/1610 /kaggle/input/visdrone2019-det-test-dev/images/0000006_01111_d_0000003.jpg: 384x640 2 pedestrians, 12 cars, 2 trucks, 1 bus, 3 motors, 8.0ms
image 4/1610 /kaggle/input/visdrone2019-det-test-dev/images/0000006_01275_d_0000004.jpg: 384x640 1 pedestrian, 13 cars, 1 truck, 1 tricycle, 2 motors, 8.0ms
image 5/1610 /kaggle/input/visdrone2019-det-test-dev/images/0000006_01659_d_0000004.jpg: 384x640 7 cars, 23 trucks, 8.4ms
image 6/1610 /kaggle/input/visdrone2019-det-test-dev/images/0000006_02138_d_0000006.jp

In [30]:
!zip -r '/kaggle/working/runs/detect/train3.zip' '/kaggle/working/runs/detect/train3'

  adding: kaggle/working/runs/detect/train3/ (stored 0%)
  adding: kaggle/working/runs/detect/train3/labels.jpg (deflated 27%)
  adding: kaggle/working/runs/detect/train3/results.png (deflated 8%)
  adding: kaggle/working/runs/detect/train3/confusion_matrix_normalized.png (deflated 18%)
  adding: kaggle/working/runs/detect/train3/val_batch2_labels.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/train3/val_batch0_pred.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/train3/val_batch2_pred.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/train3/train_batch2.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/train3/F1_curve.png (deflated 8%)
  adding: kaggle/working/runs/detect/train3/val_batch0_labels.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/train3/labels_correlogram.jpg (deflated 33%)
  adding: kaggle/working/runs/detect/train3/train_batch1.jpg (deflated 3%)
  adding: kaggle/working/runs/detect/train3/weights/ (stored 0%)
  adding: kaggle/working/run

In [ ]:
!zip -r '/kaggle/working/runs/detect/predict.zip' '/kaggle/working/runs/detect/predict'

In [33]:
!zip -r '/kaggle/working/runs/detect/val.zip' '/kaggle/working/runs/detect/val'

  adding: kaggle/working/runs/detect/val/ (stored 0%)
  adding: kaggle/working/runs/detect/val/confusion_matrix_normalized.png (deflated 18%)
  adding: kaggle/working/runs/detect/val/val_batch2_labels.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/val/val_batch0_pred.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/val/val_batch2_pred.jpg (deflated 5%)
  adding: kaggle/working/runs/detect/val/F1_curve.png (deflated 8%)
  adding: kaggle/working/runs/detect/val/val_batch0_labels.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/val/val_batch1_pred.jpg (deflated 4%)
  adding: kaggle/working/runs/detect/val/confusion_matrix.png (deflated 17%)
  adding: kaggle/working/runs/detect/val/PR_curve.png (deflated 7%)
  adding: kaggle/working/runs/detect/val/P_curve.png (deflated 4%)
  adding: kaggle/working/runs/detect/val/R_curve.png (deflated 9%)
  adding: kaggle/working/runs/detect/val/val_batch1_labels.jpg (deflated 3%)
